In [1]:
# pip install pymorphy2 >> None

In [129]:
import pandas as pd
import numpy as np
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# задаем директорию
file_path = 'C:/Users/user/Desktop/Hackathons/LinkedIn_menthoring/'

In [85]:
# открываем файлы с данными
posts = pd.read_csv(os.path.join(file_path, 'posts.csv'), index_col=0)
profiles = pd.read_csv(os.path.join(file_path, 'profiles.csv'), index_col=0)

In [86]:
# открываем датафрейм с постами
posts.head(2)

,user_id,text,likes,comments,reposts
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [87]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [88]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [89]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [90]:
# удалим посты на украинском языке

# определяем шаблон для украинских символов (по специфичным для данного языка символам)
ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# создаем маску, указывающую строки, в которых столбец "post" содержит текст на украинском языке
mask = posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)

# сохраняем в датафрейме только строки, в которых маска имеет значение False
posts = posts[~mask]

In [92]:
# сохраняем хэштэги в отдельный столбец перед их удалением из постов
posts['hashtags'] = posts['post'].str.findall(r'#([^\s]+)').apply(lambda x: ', '.join(x))

In [93]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [94]:
# удаляем слова, которые идут после хэш-тэга
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'#[^\s]+', ' ', x))

In [95]:
# производим замену дефиса на пробел
posts["post_lemmatized"] = posts["post_lemmatized"].str.replace("-", " ")

In [96]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)
# только русские буквы и пробелы
posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', ' ', regex=True) 

In [97]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
file_path_words = os.path.join(file_path, 'stop_words.txt')
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [99]:
'''если НЕ оставляем в постах английские слова'''
# определяем шаблон регулярного выражения для русских букв
pattern = '[^а-яА-ЯёЁ]'
# создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
# фильтруем датафрейм, используя маску
posts = posts[mask]

In [100]:
posts.head()

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0,"podcast, миражи",говоря подкаст мираж доступный соцсеть вконтакте разобраться туда прикрутить
2,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0,"вакансия, vacancy, illustration, иллюстрация, project, проект",честной народ искать векторный иллюстратор проект возможный длительный сотрудничество итогу удаленно уровень иллюстрация плохой пример репост пожалуйста ести уметь рисовать написать личка
3,ali-wodan,"Новый пост подкаста ""Миражи"" на всех платформах:В аудио: https://lnkd.in/giWuSVNВ инстаграме: https://lnkd.in/gbV6yxKН #automotive # # #cars #а патреоне: https://lnkd.in/gfShqTU#саморазвитие #психология #психологиячеловека #психологияжизни #психологияличности #личностныйрост #подкаст #подкастмиражи #маркетинг #осознанность #мысли #мысливслух #установки #влияние #подсознаниеИспользованные звуки, музыка и картинки:Аудиоцитата из фильма ""Формула любви"" Марка ЗахароваJardins du Luxembourg by Jahzzar is licensed under a Attribution-ShareAlike 3.0 International License.prisoner by Luis Prado from the Noun Project",1,0,0,"automotive, cars, а, саморазвитие, психология, психологиячеловека, психологияжизни, психологияличности, личностныйрост, подкаст, подкастмиражи, маркетинг, осознанность, мысли, мысливслух, установки, влияние, подсознаниеИспользованные",новый пост подкаст миражи платформах аудио инстаграме патреоне звуки музыка картинки аудиоцитат фильм формула любви марк захарова
4,ali-wodan,"Подкаст Миражи, Эпизод 13Ошибка невозвратных затратpodcast.ru/1539345144#подкаст #podcast #it #podcasts #psychology #психология #miragespodcast",2,0,0,"подкаст, podcast, it, podcasts, psychology, психология, miragespodcast",подкаст миражи эпизод ошибка невозвратный затрат
5,ali-wodan,Новый эпизод об эффекте ИКЕА на всех платформах podcast.ru/1539345144,1,0,0,,новый эпизод эффект икеа платформа


In [101]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2119 entries, 0 to 8597
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          2119 non-null   object
 1   post             2119 non-null   object
 2   likes            2119 non-null   object
 3   comments         2119 non-null   int64 
 4   reposts          2119 non-null   int64 
 5   hashtags         2119 non-null   object
 6   post_lemmatized  2119 non-null   object
dtypes: int64(2), object(5)
memory usage: 132.4+ KB


Объединяем датафреймы

In [102]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [103]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [105]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [106]:
# удаляем из столбца likes точки, запятые и пробелы
df["likes"] = df["likes"].replace(r'\.|\,|\s', '', regex=True)

# меняем тип данных сотлбца likes на integer
df["likes"] = df["likes"].astype("int64")

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2110 entries, 0 to 2118
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2110 non-null   object
 1   post              2110 non-null   object
 2   likes             2110 non-null   int64 
 3   comments          2110 non-null   int64 
 4   reposts           2110 non-null   int64 
 5   hashtags          2110 non-null   object
 6   post_lemmatized   2110 non-null   object
 7   user_name         2110 non-null   object
 8   user_head         2110 non-null   object
 9   user_work         2021 non-null   object
 10  user_position     2110 non-null   object
 11  user_tags         401 non-null    object
 12  user_location     2103 non-null   object
 13  user_viewers      2110 non-null   object
 14  user_contacts     2110 non-null   object
 15  user_common_info  1918 non-null   object
dtypes: int64(3), object(13)
memory usage: 280.2+ KB


In [108]:
# Сохраняем датафрейм
df.to_csv(os.path.join(file_path, 'linkedin.csv'))

# Сохраняем датафрейм лемматизации
df[[
    'user_id', 'post_lemmatized', 'likes', 'comments', 'reposts'
]].to_csv(os.path.join(file_path, 'post_lemmatized.csv'))

In [110]:
df.sample(5)

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
446,melekes,Находки в сети за август,0,0,0,,находка сеть август,Anton Kalyaev,"Software Engineer, Traveler, Glorified plumber #distributedsystems #consensus",Parity Technologies,Core Rust Developer,NaN,Russia,"1,183",500+,"Hi! My name is Anton. I am an experienced (10+ years) backend developer interested in distributed systems and hard problems. Ideally looking for a remote position, although relocation is possible.- “not a great programmer; just a good programmer with great habits” (Kent Beck)- have a blog http://homeonrails.com/. Popular articles include How to be productive withalmost any text editor https://www.homeonrails.com/2016/05/how-to-be-productive-with-almost-any-editor/ and Debugging the Memory Leak in Tendermint https://www.homeonrails.com/2018/03/debugging-the-memory-leak-in-tendermint/- “best April fool joke ever” author (according to my past colleagues)PROJECTS:- implemented a Merkle tree in Rust https://github.com/melekes/merkle-tree-rs- created a visualisation of the Tendermint consensus algorithm (see https://www.homeonrails.com/thesecretlivesofdata/tendermint/)- developed a simple distributed database in Erlang based on Riak Core (600 lines of code)- conducted a course on Vim editor w/ ~20 students (course materials - https://github.com/melekes/vim-is-hard-lessons)- tried to write a GUI application in Haskell during a one-night hackathon https://github.com/melekes/mike-on-linux- presented some of the lessons learned to my colleagues and community (slides on SpeakerDeck - https://speakerdeck.com/melekes)LANGUAGES AND TECHNOLOGIES:I am able to write code on a marker board in Go and Rust.At various times I wrote > 1000 lines in Go, Erlang, Rust, Ruby, Java, C, Clojure, Javascript, PHP, C# (I forgot last 3 languages).What I use in my daily work: Linux, Git, Ansible, Docker, Tmux, and Vim.Technologies: Hadoop / HBase, Postgresql, Mysql, Erlang OTP, Kafka, Redis,Jepsen, Prometheus, WebSocketsMore links:- https://gist.github.com/melekesApr 2021 - Oct 2021: I decided to take a big break to travel (hiked in the Altai mountains, drank water from Lake Baikal, hitchhiked to the south of Russia), and also spend some time with my family during the COVID-19 pandemic."
1263,yury-zhukovsky,"Всем привет!Понимать и правильно составлять SQL-запросы – важный навык для работы в QA. На w3schools.com можно отлично прокачать данный навык, а уже после изучать DBeaver.Недавно загрузил свои результаты изучения SQL на GitHub (link на репозиторий – https://lnkd.in/dW-RBipA). Выполнил 21 задание, начиная от простых запросов к более сложным.#sql #w3schools #qa #qualityassurance #opentowork #ksendzov",3,2,0,"sql, w3schools, qa, qualityassurance, opentowork, ksendzov",привет понимать правильно составлять запрос важный навык работа отлично прокачать данный навык изучать недавно загрузить результат изучение репозиторий выполнить задание начинать простой запрос сложным,Yury Zhukovsky,QA Engineer,Belarusian State University of Informatics and Radioelectronics,QA Engineer,NaN,"Minsk, Belarus","1,296",500+,"I am a Junior QA Engineer with deep knowledge of product testing that I gained while working as an online marketer for 7 years. I have big experience of working with Google's Search Quality Raters Guidelines. I have worked with tools: Jira, Google Chrome Dev Tools, GitHub, Postman, Fiddler, Charles, Linux terminal, Visual Studio Code. My key soft skills: fast learner, mathematical- analytical thinking, ability to work in a team, mindfulness, critical mindset and curiosity."
810,vladimirmoiseev,"Не про #ИмпортозамещениеТоварищи!Кто-нибудь знаком со спецификой ИТ рынка Китая? Речь про Технологические и софтверные компании второго и третьего эшелона.Что -там вообще сейчас происходит, какие тренды есть, какие сложности испытывают (кроме гос. регуляторов), прибегают ли к услуг

---

In [114]:
docs = df['post_lemmatized'].tolist()

In [116]:
tfidf = TfidfVectorizer(min_df=10, max_df=0.9)

In [117]:
x = tfidf.fit_transform(docs)

In [120]:
tfidf.fit(docs)

TfidfVectorizer(max_df=0.9, min_df=10)

In [118]:
x.shape

(2110, 1738)

In [122]:
id2word = {i: token for token, i in tfidf.vocabulary_.items()}

**LDA**

In [123]:
n_topics = 20

In [124]:
lda = LatentDirichletAllocation(n_components=n_topics, random_state=31)

In [125]:
topics = lda.fit_transform(x)

In [126]:
topics.shape

(2110, 20)

**Ключевые слова**

In [130]:
for i in range(n_topics):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

Тема 1 :
школа, шаблон, всякий, анимация, визуальный, поделитесь, сделать, занять, хотеть, сменить, учёт, хороший, заказ, продвижение, ловить, модель, яндекс, презентация, скидка, лента

 
Тема 2 :
пишите, сфере, приглашать, официальный, гость, связаться, алматы, компетенция, карточка, картина, экономия, нанимать, учить, продажа, круто, сети, игровой, рада, связь, обратный

 
Тема 3 :
статья, оценить, формировать, обеспечивать, крутой, разбирать, успеть, отрасль, команда, запись, сэкономить, встретить, супер, надеюсь, разработчик, пропустить, удалённый, офис, подробность, облачный

 
Тема 4 :
функциональный, желательно, программирования, самостоятельный, команда, отличный, плата, участвовать, член, командный, апрель, заработный, очень, запустить, вклад, ценить, умение, благодарность, сеть, находка

 
Тема 5 :
программист, приглашать, интенсив, нуль, краткий, объяснять, абсолютно, многий, слово, руководство, бот, реклама, аудитория, программирование, ожидать, успешно, достигнуть, заметк

**Типичные статьи**

In [133]:
for i in range(n_topics):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i)
    print(df.iloc[doc_id]["post"])
    print("\n")

Тема  0
Redesign presentation of Yandex company and I demonstrated how to improve the visual side of the presentation. The animation was done entirely in Power Point.You can see the detailed process in the https://lnkd.in/d9NttCHaWould you like to place an order? You can write to Linkedin or Whatsapp https://wa.me/+77058042545#presentation #infographics #3d #abinur #googleslides #powerpoint #yandex Yandex==========================Редизайн презентации компании Яндекс и я продемонстрировал как улучшить визуальную сторону презентации. Анимация полностью сделана в Power Point.Подробный процесс вы можете посмотреть на странице https://lnkd.in/d9NttCHa.вы хотели бы сделать заказ? Вы можете написать в Facebook или Whatsapp https://wa.me/+77058042545


Тема  1
Приглашаем тебя на долгожданный офлайн митап от Product Bee Тонкости технического интервьюНавыки в разработке продуктаБесплатные ресурсы для продактов Будем проводить интерактивные игрыЖдем тебя 13 мая 18:00 | Jas Coworking | Алматы.Мест